# finetune a LLM

## setup

In [1]:
!python --version

Python 3.10.11


In [2]:
from __future__ import print_function
import sys, os
print(sys.path)

['/Users/brucecottman/Documents/pyProjects/ftLLM/notebooks', '/usr/local/Cellar/apache-spark/2.0.1/libexec/python', '/Users/brucecottman/Documents/pyProjects/ftLLM/notebooks/$', '/usr/local/Cellar/python@3.10/3.10.11/Frameworks/Python.framework/Versions/3.10/lib/python310.zip', '/usr/local/Cellar/python@3.10/3.10.11/Frameworks/Python.framework/Versions/3.10/lib/python3.10', '/usr/local/Cellar/python@3.10/3.10.11/Frameworks/Python.framework/Versions/3.10/lib/python3.10/lib-dynload', '', '/Users/brucecottman/Documents/pyProjects/ftLLM/venv/lib/python3.10/site-packages', '/usr/local/lib/python3.10/site-packages']


In [3]:
%load_ext autoreload
%autoreload 2
packages = !conda list
packages

['# packages in environment at /Users/brucecottman/anaconda3:',
 '#',
 '# Name                    Version                   Build  Channel',
 'abseil-cpp                20211102.0           h96cf925_1    conda-forge',
 'alabaster                 0.7.12             pyhd3eb1b0_0  ',
 'altair                    4.2.2              pyhd8ed1ab_0    conda-forge',
 'anaconda-client           1.11.2          py310hecd8cb5_0  ',
 'anaconda-navigator        2.4.0           py310hecd8cb5_0  ',
 'anaconda-project          0.11.1          py310hecd8cb5_0  ',
 'anyio                     3.5.0           py310hecd8cb5_0  ',
 'appdirs                   1.4.4              pyhd3eb1b0_0  ',
 'applaunchservices         0.3.0           py310hecd8cb5_0  ',
 'appnope                   0.1.2           py310hecd8cb5_1001  ',
 'appscript                 1.1.2           py310hca72f7f_0  ',
 'argon2-cffi               21.3.0             pyhd3eb1b0_0  ',
 'argon2-cffi-bindings      21.2.0          py310hca72f7f_0  '

In [91]:
!pip list


Package                  Version
------------------------ --------
aiohttp                  3.8.4
aiosignal                1.3.1
anyio                    3.6.2
appnope                  0.1.3
argon2-cffi              21.3.0
argon2-cffi-bindings     21.2.0
arrow                    1.2.3
asttokens                2.2.1
async-timeout            4.0.2
attrs                    23.1.0
backcall                 0.2.0
beautifulsoup4           4.12.2
bleach                   6.0.0
certifi                  2023.5.7
cffi                     1.15.1
charset-normalizer       3.1.0
comm                     0.1.3
dataclasses-json         0.5.7
debugpy                  1.6.7
decorator                5.1.1
defusedxml               0.7.1
executing                1.2.0
fastjsonschema           2.16.3
fqdn                     1.5.1
frozenlist               1.3.3
greenlet                 2.0.2
html5lib                 1.1
idna                     3.4
ipykernel                6.23.1
ipython                  8.1

## pdf to text

 a Python function that can be used to read in a PDF using the PyMuPDF package and output a list of texts where each text is the text inside each subsection. The function also outputs a dictionary where each property is the subsection and the values of the property are subsection level and the text inside each subsection. The function is type annotated and documented using the Google doc-string style:

In [1]:
from typing import Union, Tuple, Dict, List
import fitz
import os

def extract_subsections(pdf_paths: Union[str, List[str]]) -> \
                        Tuple[int, Dict[str, Tuple[int, str]], str]:
    """
    Extracts the texts inside each subsection of one or more PDFs 
    and also builds a dictionary with subsections as keys
    and a tuple of subsection level and subsection text as values.

    Args:
        pdf_paths (Union[str, List[str]]): The path(s) to the PDF file(s).

    Returns:
        Tuple[int, Dict[str, Tuple[int, str]], str]: A tuple with the 
        total number of extracted subsection texts, a dictionary where 
        each property is the subsection, and the values of the property 
        are subsection level and the text inside each subsection, 
        and a text corpus str these texts.

    Raises:
        RuntimeError: If the file(s) specified by pdf_paths do not exist or cannot be opened.
    """

    
    if not isinstance(pdf_paths, list):
        pdfs = [pdf_paths]
    else:
        pdfs = pdf_paths

    section_texts = []
    section_info = {}
        
    for pdf_path in pdfs:
        # Error handling check
        if not os.path.isfile(pdf_path):
            raise FileNotFoundError(f"No such file or directory: '{pdf_path}'")
        
        try:
            doc = fitz.open(pdf_path) 
        except RuntimeError as e:
            raise RuntimeError(f"Failed to open file '{pdf_path}': {e}")

        for page in doc:
            blocks = page.get_text("blocks")
            for block in blocks:
                text = block[4]
                section_texts.append(text)
                lines = text.splitlines()
                for line in lines:
                    if line.endswith(':'):  
                        level = len(line.split())
                        section_info[line[:-1]] = (level, '\n'.join(lines[lines.index(line) + 1:]))

    return len(section_texts), section_info, " ".join(section_texts)

In [152]:
# extract_subsections(["../PDFs/Text and Code Embeddings.pdf",
#                     "../PDFs/Place_Freeze_Success_PDF.pdf",
#                     "../PDFs/0a6323ba-1642-45a6-9152-cd6eaaf9b4b0 (1).pdf",
#                     ])

In [191]:
corpus_lists_length,section_info,corpus_lists= extract_subsections([
                    "../PDFs/Place_Freeze_Success_PDF.pdf",
                    "../PDFs/0a6323ba-1642-45a6-9152-cd6eaaf9b4b0 (1).pdf",
                    "../PDFs/Text and Code Embeddings.pdf",
                    ])
print('corpus_lists_length=',corpus_lists_length)
for i in corpus_lists:
    print(i)
    print("\n")

section_info

n= 286
Text and Code Embeddings by Contrastive Pre-Training



Arvind Neelakantan * 1 Tao Xu * 1 Raul Puri 1 Alec Radford 1 Jesse Michael Han 1 Jerry Tworek 1



Qiming Yuan 1 Nikolas Tezak 1 Jong Wook Kim 1 Chris Hallacy 1 Johannes Heidecke 1 Pranav Shyam 1



Boris Power 1 Tyna Eloundou Nekoul 1 Girish Sastry 1 Gretchen Krueger 1 David Schnurr 1



Felipe Petroski Such 1 Kenny Hsu 1 Madeleine Thompson 1 Tabarak Khan 1 Toki Sherbakov 1 Joanne Jang 1



Peter Welinder 1 Lilian Weng 1



Abstract



Text embeddings are useful features in many
applications such as semantic search and com-
puting text similarity. Previous work typically
trains models customized for different use cases,
varying in dataset choice, training objective and
model architecture. In this work, we show that
contrastive pre-training on unsupervised data at
scale leads to high quality vector representations
of text and code. The same unsupervised text em-
beddings that achieve new state-of-the-art results
in linear-p

.

## tranform list of test into sub-strings  is less than 1500 words.

Please note that each file path should not contain spaces, or if they do, the paths should be quoted, like -pdf "file 1.pdf" "file 2.pdf".

In [2]:
import fitz
from typing import List, Dict, Tuple, Union

def extract_subsections(pdf_paths: Union[str, List[str]]) ->\
                        Tuple[int, Dict[str, Tuple[int, str]], List[str]]:
    """
    Extracts the texts inside each subsection of a PDF 
    and also builds a dictionary with 
    subsections as keys and a tuple of subsection level 
    and subsection text as values.

    Args:
        pdf_paths (Union[str, List[str]]): The path(s) to the PDF file(s).

    Returns:
        Tuple[int, Dict[str, Tuple[int, str]], List[str]]: A tuple with 
        the total number of extracted subsection texts, a dictionary 
        where each property is the subsection and the values of the 
        property are subsection level and the text inside 
        each subsection, and a list of these texts.
    """
    
    if not isinstance(pdf_paths, list):
        pdf_paths = [pdf_paths]

    section_texts = []
    section_info = {}
        
    for pdf_path in pdf_paths:
        doc = fitz.open(pdf_path) 

        for page in doc:
            blocks = page.get_text("blocks")
            for block in blocks:
                text = block[4]
                section_texts.append(text)
                lines = text.splitlines()
                for line in lines:
                    if line.endswith(':'):  
                        level = len(line.split())
                        section_info[line[:-1]] = (level, '\n'.join(lines[lines.index(line) + 1:]))

    return len(section_texts), section_info, section_texts

## contractions

In [3]:
contractions = {
    "'bout": "about",
    "'cause": "because",
    "'em": "them",
    "'til": "until",
    "'ve": "have",
    "ain't": "am not",
    "aren't": "are not",
    "can't": "cannot",
    "couldn't": "could not",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "e'er": "ever",
    "gonna": "going to",
    "gotta": "got to",
    "hadn't": "had not",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he had",
    "he'll": "he will",
    "he's": "he is",
    "here's": "here is",
    "how'd": "how did",
    "how'll": "how will",
    "how's": "how is",
    "I'd": "I had",
    "I'll": "I will",
    "I'm": "I am",
    "I've": "I have",
    "isn't": "is not",
    "it'd": "it had",
    "it'll": "it will",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mustn't": "must not",
    "needn't": "need not",
    "o'er": "over",
    "shan't": "shall not",
    "she'd": "she had",
    "she'll": "she will",
    "she's": "she is",
    "shouldn't": "should not",
    "so've": "so have",
    "so's": "so is",
    "that'd": "that had",
    "that'll": "that will",
    "that's": "that is",
    "there'd": "there had",
    "there'll": "there will",
    "there's": "there is",
    "they'd": "they had",
    "they'll": "they will",
    "they're": "they are",
    "they've": "they have",
    "this'll": "this will",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we had",
    "we'll": "we will",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'd": "what did",
    "what'll": "what will",
    "what's": "what is",
    "when'd": "when did",
    "when'll": "when will",
    "when's": "when is",
    "where'd": "where did",
    "where'll": "where will",
    "where's": "where is",
    "who'd": "who had",
    "who'll": "who will",
    "who's": "who is",
    "why'd": "why did",
    "why'll": "why will",
    "why's": "why is",
    "won't": "will not",
    "wouldn't": "would not",
    "y'all": "you all",
    "you'd": "you had",
    "you'll": "you will",
    "you're": "you are",
    "you've": "you have"
}

In [5]:
from typing import Dict, List

def expand_contractions(text: str, contractions: Dict[str, str]) -> str:
    """
    Expands contractions in a corpus text.

    Args:
        text (str): The corpus text.
        contractions (Dict[str, str]): A dictionary that maps contractions to their expanded forms.

    Returns:
        str: The expanded corpus text.
    """
    expanded_text = []
    for word in text.split():
        if word in contractions:
            expanded_text.append(contractions[word])
        else:
            expanded_text.append(word)

    return " ".join(expanded_text)


## cleaning functions

In [7]:
"""
This module provides functions for cleaning text.

Functions:

* remove_numbers: Removes numbers from the input text.
* remove_urls: Removes URLs from the input text.
* remove_emails: Removes email addresses from the input text.

* to_lowercase: Converts the input text to lowercase.

"""

from typing import List, Tuple

import spacy
import re


nlp = spacy.load('en_core_web_sm')


def remove_numbers(text: str) -> str:
    """Removes numbers from the input text.

    Args:
    text (str): Text from which to remove numbers.

    Returns:
    str: Text without numbers.
    """

    doc = spacy.load("en_core_web_sm")(text)
    return "".join([token.text for token in doc if not token.is_digit])

def remove_html_tags(text: str) -> str:
    """
    Removes HTML tags from a text string using regular expressions.

    Args:
    text (str): The text to clean.

    Returns:
    str: The text with HTML tags removed.

    Raises:
    ValueError: If the input text is not a string.

    Note: 
    The remove_html_tags function uses a basic regular 
    expression to remove HTML tags from the text. 
    This should work for most HTML, but could fail 
    in some edge cases (such as comments or scripts embedded within the HTML). 
    For more complex HTML parsing and cleaning, 
    consider using a library designed for that purpose, like BeautifulSoup.
    """

    if not isinstance(text, str):
        raise ValueError("The input text must be a string.")

    return re.sub(r"<[^>]+>", "", text)


def remove_urls(text: str) -> str:
    """Removes URLs from the input text.

    Args:
    text (str): Text from which to remove URLs.

    Returns:
    str: Text without URLs.
    """

    return re.sub(r"http\S+|www\.\S+", "", text)


def remove_emails(text: str) -> str:
    """Removes email addresses from the input text.

    Args:
    text (str): Text from which to remove email addresses.

    Returns:
    str: Text without email addresses.
    """

    return re.sub(r"\S+@\S+", "", text)

def remove_punctuation(text: str) -> str:
    """
    Removes punctuation from the input text.

    Args:
        text (str): Text from which to remove punctuation.

    Returns:
        str: Text without punctuation.
    """
    doc = nlp(text)
    return ''.join([token.text for token in doc if not token.is_punct])

def remove_stopwords(text: str) -> str:
    """
    Removes stopwords from the input text.

    Args:
        text (str): Text from which to remove stopwords.

    Returns:
        str: Text without stopwords.
    """
    doc = nlp(text)
    return ' '.join([token.text for token in doc if not token.is_stop])

def to_lowercase(text: str) -> str:
    """Converts the input text to lowercase.

    Args:
    text (str): Text to convert to lowercase.

    Returns:
    str: Text in lowercase.
    """

    return text.lower()


def tokenize_text(text: str) -> List[str]:
    """
    Tokenizes the input text.

    Args:
        text (str): Text to tokenize.

    Returns:
        List[str]: List of tokens.
    """
    doc = nlp(text)
    return [token.text for token in doc]

def lemmatize_text(text: str) -> List[str]:
    """
    Lemmatizes the input text.

    Args:
        text (str): Text to lemmatize.

    Returns:
        List[str]: List of lemmas.
    """
    doc = nlp(text)
    return [token.lemma_ for token in doc]



IndentationError: expected an indented block after 'if' statement on line 58 (1973050613.py, line 59)